In [2]:
import numpy as np
import sys
if "../" not in sys.path:
    sys.path.append('../')
from lib.envs.gridworld import GridworldEnv

In [3]:
env = GridworldEnv()

In [6]:
print(env.nA)

4


### PredictとControlの違い

状態価値関数$V$のみを求めるものをPredictという

状態価値関数$V$と政策$\pi$を求めるものをControlという

### Policy Evaluation

In [32]:
# Problem: evaluate a given policy pi
# Solution: iterative application of Bellman expectation backup
# v1 \to v2 \to \cdots \to v\pi
# Using synchronous backups,
#   At each iteration k + 1
#   For all state s \in S
#   Update v_{k+1}(s) from v_k(s')
#   where s' is a successor state of s

# action = env.action_space.sample()
# observation, reward, done, info = env.step(action)

def policy_eval(policy, env, gamma=1., eps=1e-3):
    V = np.zeros(env.nS)
    while True:
        Vk = V.copy()
        for s in range(env.nS):
            v = 0
            for a in range(env.nA):
                # P[s][a] == [(probability, nextstate, reward, done), ...]
                for prob, next_s, reward, done in env.P[s][a]:
                    # v += policy[a] * (reward + gamma  * prob * V[next_s])
                    # \pi(a|s)は, sありきのaの確率だから気をつけて
                    v += policy[s, a] * (reward + gamma  * prob * V[next_s])
            V[s] = v
        if np.abs(sum(V - Vk)) < eps:
            break
    return V

In [30]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
V = policy_eval(random_policy, env)
print(V.reshape((4, 4)))

[[  0.         -13.99942284 -19.99917026 -21.99908672]
 [-13.99942284 -17.99929175 -19.99923101 -19.9992398 ]
 [-19.99917026 -19.99923101 -17.99935111 -13.99951553]
 [-21.99908672 -19.9992398  -13.99951553   0.        ]]


In [31]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(V, expected_v, decimal=2)

In [27]:
print(np.ones([env.nS, env.nA]) / env.nA)

[[ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]]


### Improve Policy

- Evalue the policy $\pi$
$$v_{\pi}(s) = \mathbb{E}[R_{t+1} + \gamma R_{t+2} + \cdots | S_{t} = s]$$

- Improve the policy by acting greedily with respect to $v_{\pi}$
$$\pi^{'} = \text{greedy}(v_{\pi})$$

- in Small gridworld improved policy was optimal, $\pi^{'} = \pi^{*}$
- In general, need more iterations of improvement / evaluation

### Policy Improvement

- Consider a deterministic policy, $a = \pi(s)$
- We can improve the policy by acting greedily
$$\pi^{'}(s) = arg \max_{a \in A}q_{\pi}(s, a)$$

- This improves the value from any state $s$ over one step,
$$q_{\pi}(s, \pi^{'}(s)) = \max_{a \in A}q_{\pi}(s, a) \geq q_{\pi}(s, \pi(s)) = v_{\pi}(s)$$

- Stop
$$v_{\pi}(s) = \max_{a \in A} q_{\pi}(s, a)$$

アイデアとしては, `action_values[a]`という`Q(s,a)`をもつ.    
そうすることによって, `choice_a = np.argmax(policy[s])`という選ばれた`s`で最も良い`a`とくらべて,    
計算された $$\text{action_values[a]} = \sum_{a \in \mathcal{A}}  [R + \gamma \sum_{s \in \mathcal{S}}V[s']]$$
が必要になる

In [60]:
random_policy = np.ones([env.nS, env.nA]) / env.nA

def policy_iteration(policy, env, gamma=1., eps=1e-3):
    Q = np.zeros([env.nS, env.nA])
    while True:
        V = policy_eval(policy, env, gamma, eps)
        
        # Will be set to false if we make any change to the policy
        policy_stable = True
        
        for s in range(env.nS):
            # The best action we would take under the currect policy
            chosen_a = np.argmax(policy[s])

            # Find the best action by one-step lookahead
            # Ties are resolved arbitarily
            action_values = np.zeros(env.nA)
            for a in range(env.nA):
                for prob, next_s, reward, done in env.P[s][a]:
                    # これが何かイマイチわかってない
                    action_values[a] += reward + gamma * prob * V[next_s]
            best_a = np.argmax(action_values)
            
            # Greedily update the policy
            # 最も良いものでなければ終了しない.
            # 最も良いものを更新している.
            if best_a != chosen_a:
                policy_stable = False
            policy[s] = np.eye(env.nA)[best_a]
        if policy_stable:
            break
    return V, policy

In [56]:
V, policy = policy_iteration(random_policy, env)

In [57]:
np.reshape(np.argmax(policy, axis=1), env.shape)

array([[0, 3, 3, 2],
       [0, 0, 0, 2],
       [0, 0, 1, 2],
       [0, 1, 1, 0]])

In [58]:
policy

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.]])

### Value Iteration

- 問題: 最適政策$\pi$を見つける
- 解決策: Bellman最適バックアップの応用反復
- $v_1 \to v_2 \cdots \to v_{*}$
- 同期バックアップを使う
    - $k+1$まで反復する
    - すべての状態$s \in \mathcal{S}$
    - $v_{k}(s')$から$v_{k+1}(s)$を更新する
    
$$v_{k+1}(s) = \max_{a \in \mathcal{A}} \bigl( \mathcal{R_{s}^{a}} + \gamma \sum_{s' \in \mathcal{S}} \mathcal{P}_{ss'}^{a} v_{k}(s') \bigr)$$

`A = one_step_lookahead(s, V)`が
$$\pi^{'}(s) = arg \max_{a \in A}q_{\pi}(s, a)$$

In [97]:
def value_iteration(env, theta=0.0001, discount_factor=1.):
    """
    Value Iteration Algo.
    
    Return:
        A tuple (policy, V) of the optimal policy and the optimal value function.
    """
    
    def one_step_lookahead(s, V):
        """
        Helper function to calc the value for all action in a given state.
        
        """
        action_vals = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_s, reward, done in env.P[s][a]:
                action_vals[a] += reward + discount_factor *  (prob * V[next_s])
        return action_vals
    
    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    
    while True:
        delta = 0
        bef_V = V.copy()
        for s in range(env.nS):
            v = one_step_lookahead(s, V)  
            """          
            v = []
            for a, action_prob in enumerate(policy[s]):
                v0 = 0
                for prob, next_s, reward, done in env.P[s][a]:
                    v0 += reward + discount_factor * (prob * V[next_s])
                v.append(v0)
            """
            delta = max(delta, np.abs(max(v) - V[s]))
            V[s] = np.max(np.asarray(v))
        if delta < theta:
            break
            
    for s in range(env.nS):
        A = one_step_lookahead(s, V)
        best_action = np.argmax(A)
        policy[s] = np.eye(env.nA)[best_action]
            
    return policy, V

In [98]:
policy, V = value_iteration(env)

In [99]:
print(V.reshape(env.shape))

[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]


In [100]:
print(policy)

[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]


In [101]:
np.reshape(np.argmax(policy, axis=1), env.shape)

array([[0, 3, 3, 2],
       [0, 0, 0, 2],
       [0, 0, 1, 2],
       [0, 1, 1, 0]])